In [1]:
import torch
from torch import nn
import numpy as np
import lgg_model
from lgg_model import softmax
from gensim.models import Word2Vec

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
lgg_model_path = input("请输入想使用的语言模型(无需添加后缀)：")
lgg_model_path = 'lgg_model_paths/' + lgg_model_path
lgg_model = torch.load(lgg_model_path)
lgg_model.eval()

vanilla_LSTM(
  (Embedding): Embedding(709, 100)
  (LSTM): LSTM(100, 100, num_layers=4, batch_first=True)
  (Linear): Linear(in_features=100, out_features=709, bias=True)
)

In [3]:
word_model_path = input("请使用想使用的词汇库(无需添加后缀)：")
word_model_path = 'word_model_paths/' + word_model_path
word_model = Word2Vec.load(word_model_path)
wv = word_model.wv

In [4]:
words = input("请输入初始文本：")
del_lst = []
lst = list(words)

for i in lst:
    if i not in wv.key_to_index:
        del_lst.append(i)
for i in del_lst:
    lst.remove(i)

data = np.array([])
for i in lst:
    data = np.append(data, wv.key_to_index[i])

count = int(input("请输入想要生成的字(词)数："))

for i in lst:
    print(i, end='')

for i in range(count):
    data = np.stack((data,))
    x = torch.Tensor(data)
    x = x.to(torch.long)
    y = lgg_model(x)[0][-1]
    p = y.detach().numpy()
    p = softmax(p)

    idx = np.random.choice(np.arange(len(wv)), p=p)
    new_word = wv.index_to_key[idx]
    print(new_word, end='')

    lst.append(new_word)
    data = np.append(data, idx)

print('\nGeneration finished.')



帝若兮带上萧！
带长剑兮，首身离兮心不惩。
诚既兮又以武，终强兮不可凌。
身既死兮神兮剑夜鸣夜之遗介兮，心日丘之善恶？
民总美之菹艰兮，非余心之所。
老冉冉其将至兮，恐修名之不立。
朝饮木兰之坠兮，夕秋菊之落英。
苟余情其信姱以作？怨将不群兮，伤独马此先桑。
济皇翼其杂路兮，及修行兮。
广立而心后以远兮，路观极兮下极。
余兮浩甲。
高美之可。
望三五以为兮，指彭咸以为。
夫何极而不至兮，故南人之知其相接。
众不可户说兮，孰云察余之善恶？
民好恶其不同兮，惟此党人其独异！
户服艾以盈要兮，谓幽兰其不可佩。
览谗乎！
忠何罪以遇兮，亦非余心之所志。
行不群以兮，又众兆之所也。
壹心而不豫兮，羌不可保也。
时於天结而诒。
蹇蹇之烦冤兮，而不发。
申以舒中情兮，志沉而莫达。
愿言於浮云兮，遇丰隆而不将。
因归鸟而致辞兮，羌高而难当。
高辛之灵兮，遭玄鸟而致诒。
欲变节以从俗兮，易初而屈志。
独历年而离愍兮，羌心犹未化。
宁发郁其若杂与君兮何不惩。
诚既兮又宜，子慕予兮善。
乘赤兮从文，辛夷车兮结桂旗。
被石兰兮带杜衡，折芳兮遗所思。
独亦余独美其变易兮，又何可以淹留？
兰芷变而不芳兮，亦前世而无
Generation finished.
